In [1]:
import os

In [2]:
%pwd

'/Users/arpansethi/Documents/ai-projects/kidney-disease/Kidney-Disease-Classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/arpansethi/Documents/ai-projects/kidney-disease/Kidney-Disease-Classification'

In [5]:
import dagshub
dagshub.init(repo_owner='arpansethi30', repo_name='Kidney-Disease-Classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as arpansethi30

Initialized MLflow to track repo "arpansethi30/Kidney-Disease-Classification"

Repository arpansethi30/Kidney-Disease-Classification initialized!

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Kidney_Disease",
            mlflow_uri="https://dagshub.com/arpansethi30/Kidney-Disease-Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [20]:
import mlflow
import dagshub
from urllib.parse import urlparse
import tempfile
import os


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


    def log_into_mlflow(self):
        # Initialize DagsHub
        dagshub.init(repo_owner='arpansethi30', repo_name='Kidney-Disease-Classification', mlflow=True)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            # Save the model to a temporary directory
            with tempfile.TemporaryDirectory() as tmp_dir:
                model_path = os.path.join(tmp_dir, "model")
                self.model.save(model_path)
                
                # Log the model as an artifact
                mlflow.log_artifact(model_path, "model")
            
            # Register the model if not using file store
            if tracking_url_type_store != "file":
                mlflow.register_model(f"runs:/{mlflow.active_run().info.run_id}/model", "VGG16Model")

        print("Model logged successfully in MLflow")

In [21]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-10-04 01:43:52,210: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-10-04 01:43:52,212: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-04 01:43:52,213: INFO: common: created directory at: artifacts]
Found 2207 images belonging to 2 classes.
138/138 [==============================] - 92s 666ms/step - loss: 1.7061 - accuracy: 0.7209
[2024-10-04 01:45:24,379: INFO: common: json file saved at: scores.json]
[2024-10-04 01:45:24,914: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/arpansethi30/Kidney-Disease-Classification "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "arpansethi30/Kidney-Disease-Classification"

[2024-10-04 01:45:24,926: INFO: helpers: Initialized MLflow to track repo "arpansethi30/Kidney-Disease-Classification"]


Repository arpansethi30/Kidney-Disease-Classification initialized!

[2024-10-04 01:45:24,928: INFO: helpers: Repository arpansethi30/Kidney-Disease-Classification initialized!]
INFO:tensorflow:Assets written to: /var/folders/hv/392wjqgd1j34prxvqfc8zwwc0000gn/T/tmpznrdd6uy/model/assets
[2024-10-04 01:45:26,903: INFO: builder_impl: Assets written to: /var/folders/hv/392wjqgd1j34prxvqfc8zwwc0000gn/T/tmpznrdd6uy/model/assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/10/04 01:45:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.


Model logged successfully in MLflow


In [22]:
import tensorflow as tf
print(tf.__version__)
import mlflow
print(mlflow.__version__)

2.13.0
2.2.2
